In [42]:
import pymongo
import pandas as pd
import numpy as np
import re
import time
import datetime
import nltk
from datetime import datetime, timedelta
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from bson.objectid import ObjectId

In [44]:
client = pymongo.MongoClient('localhost', 27017)
db = client.db
threads = db.threads
companies = db.companylist
matches = db.matches
chunks = db.chunks
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/user/nltk_data...


True

In [45]:
df = pd.DataFrame.from_records(threads.find({'Label':{'$ne': 0}}))

In [46]:
df

,_id,Inserted Date,Title,Body,Tag,Label,Stocks
0,5f766db0c41ae878a718020e,2020-10-01 23:18:55.840,List of companies that are fucked next year,For those who aren t aware new york is going i...,DD,bearish,"[$KIM, $CXP, $CLI, $AKR, $BRX, CLI, Piedmont O..."
1,5f766db0c41ae878a7180210,2020-10-01 23:18:55.840,VIRGIN GALACTIC ROCKET SHIPS ARE COOL PT $180 ...,VIRGIN GALACTIC ROCKET SHIPS ARE COOL PT $180...,DD,bullish,"[Virgin Galactic Holdings, Inc., SPCE, $SPCE, ..."
2,5f766db0c41ae878a7180212,2020-10-01 23:18:55.840,Updated S P500 Technical Analysis Bears Are St...,Last week I shared my S P500 forecast and brok...,Technicals,bullish,"[Moderna, Inc., SPY, $SPY]"
3,5f766db0c41ae878a7180215,2020-10-01 23:18:55.840,Procter Gamble to the fucking MOON DD inside,Procter Gamble 10/27/2020 Q1 Earnings Play NY...,DD,bullish,"[Procter & Gamble Company (The), TSLA, PG, $TS..."
4,5f766db0c41ae878a7180217,2020-10-01 23:18:55.840,WHY IS NO ONE TALKING ABOUT BABA TODAY,This sub introduced me to BABA. I am BABA gang...,Discussion,bullish,"[$BABA, BABA]"
...,...,...,...,...,...,...,...
75,5f7787fc1d41cb76fd11304c,2020-10-02 20:05:16.466,Is it time to bull,Rona got Orange. What are you thinking guys I ...,Discussion,bullish,"[Nasdaq, Inc.]"
76,5f7787fc1d41cb76fd11304e,2020-10-02 20:05:16.466,CHWY go go go,CHWY LISTEN UP RETARDS chwy was moon for the p...,YOLO,bullish,"[CHWY, $CHWY]"
77,5f7b395a6d0682ca2381a86f,2020-10-05 15:18:50.943,AMD going to $100 This Month,I m looking at AMD Technicals https //imgur.co...,Technicals,bullish,"[TTM Technologies, Inc., TTM, $AMD, AMD]"
78,5f7b395a6d0682ca2381a870,2020-10-05 15:18:50.943,Actual Second Wave,Back in early summer this year the US had a se...,DD,bearish,"[SPY, $SQQQ, SQQQ, $SPY]"


In [47]:
sir = SentimentIntensityAnalyzer()

In [58]:
sent = df.loc[77,'Title'] + ' ' + df.loc[77,'Body']

In [59]:
sir.polarity_scores(sent)['compound']

0.9459

In [65]:
for i in range(len(df)):
    sent = df.loc[i,'Title'] + ' ' + df.loc[i,'Body']
    score = sir.polarity_scores(sent)['compound']
    if (score < 0 and df.loc[i, 'Label'] == 'bullish') or (score > 0 and df.loc[i,'Label'] == 'bearish'):
        print(str(score) + ':' + df.loc[i, 'Label'])

-0.9077:bullish
-0.7453:bullish
-0.4522:bullish
-0.9222:bullish
0.9274:bearish
0.8258:bearish
0.1027:bearish
-0.6037:bullish
-0.9495:bullish
0.8022:bearish
-0.5267:bullish
-0.5994:bullish
0.3716:bearish
0.8689:bearish
-0.743:bullish
0.4791:bearish
-0.608:bullish
0.2263:bearish
-0.8371:bullish
-0.8185:bullish
0.6326:bearish
0.9801:bearish


In [67]:
58/80

0.725

In [69]:
import flair
from flair.models import TextClassifier
from flair.data import Sentence

In [71]:
classifier = TextClassifier.load('en-sentiment')
sentence = Sentence(sent)
classifier.predict(sentence)

2020-10-28 20:58:22,625 https://nlp.informatik.hu-berlin.de/resources/models/sentiment-curated-distilbert/sentiment-en-mix-distillbert_3.1.pt not found in cache, downloading to /tmp/tmpmkxr9cg5
100%|██████████| 266147697/266147697 [22:28<00:00, 197411.37B/s]2020-10-28 21:20:51,350 copying /tmp/tmpmkxr9cg5 to cache at /home/user/.flair/models/sentiment-en-mix-distillbert_3.1.pt

2020-10-28 21:20:52,300 removing temp file /tmp/tmpmkxr9cg5
2020-10-28 21:20:52,336 loading file /home/user/.flair/models/sentiment-en-mix-distillbert_3.1.pt


In [73]:
sentence.labels[0]

NEGATIVE (1.0)

In [76]:
for i in range(len(df)):
    sent = df.loc[i,'Title'] + ' ' + df.loc[i,'Body']
    sentence = Sentence(sent)
    classifier.predict(sentence)
    print(df.loc[i,'Label'] + ': ' + str(sentence.labels[0]))

bearish: NEGATIVE (0.9999)
bullish: NEGATIVE (0.8538)
bullish: NEGATIVE (1.0)
bullish: POSITIVE (0.9986)
bullish: POSITIVE (0.6076)
bullish: NEGATIVE (0.9998)
bullish: NEGATIVE (0.9994)
bullish: NEGATIVE (0.9999)
bullish: NEGATIVE (0.6174)
bullish: NEGATIVE (0.9998)
bearish: NEGATIVE (1.0)
bullish: POSITIVE (0.849)
bullish: NEGATIVE (0.9989)
bullish: NEGATIVE (0.9463)
bullish: NEGATIVE (0.859)
bullish: NEGATIVE (0.9999)
bearish: NEGATIVE (0.9998)
bullish: NEGATIVE (0.9995)
bearish: NEGATIVE (1.0)
bearish: NEGATIVE (1.0)
bullish: POSITIVE (0.9216)
bullish: NEGATIVE (0.9955)
bullish: NEGATIVE (0.9999)
bearish: NEGATIVE (0.9983)
bullish: NEGATIVE (1.0)
bullish: POSITIVE (0.8317)
bullish: POSITIVE (0.8583)
bullish: NEGATIVE (0.9385)
bullish: NEGATIVE (0.9987)
bullish: NEGATIVE (0.9731)
bullish: NEGATIVE (1.0)
bullish: NEGATIVE (0.9986)
bullish: NEGATIVE (0.9996)
bullish: NEGATIVE (0.9864)
bullish: NEGATIVE (1.0)
bullish: NEGATIVE (1.0)
bullish: POSITIVE (0.9995)
bullish: NEGATIVE (1.0)
bea